In [4]:
import numpy as np
import cv2
import os
import sys
import math
import random
import copy
import inspect
from matplotlib import pyplot as plot
import pandas as pd
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.filters.rank import equalize
from skimage.morphology import disk

### Remarque : 
#### Le préprocessing des données de la dataset complète a deja été fait , il coutera cher en temps de calcul de le refaire ici ( 5h ~ 7h ), c'est pour cela que je vous propose de ne pas executer les cellules de code qui font le préprocessing des données et de se contenter de la cellule qui charge les données prétraitées.

### Fonction : locate_iris_boundaries
Détecte dynamiquement les limites internes et externes de l'iris en ajustant progressivement les paramètres de recherche en fonction des résultats initiaux.
(La détection est effectuée dans d'autres fonctions. Cette fonction utilise les autres fonctions afin de détecter les bordures et ajuste les paramètres jusqu'à ce que les limites soient trouvées.)

In [5]:
def locate_iris_boundaries(image, display=False):
    inner_circle = detect_inner_circle(image)

    if not inner_circle:
        print('ERROR: Inner circle not found!')
        return None, None

    radius_expansion = int(math.ceil(inner_circle[2]*1.5))
    expansion_factor = 0.25
    range_center = int(math.ceil(inner_circle[2]*expansion_factor)) 
    outer_circle = detect_outer_circle(
                        image, inner_circle, range_center, radius_expansion)

    while(not outer_circle and expansion_factor <= 0.7):
        expansion_factor += 0.05
        print('Searching outer iris circle with expansion factor ' + str(expansion_factor))

        range_center = int(math.ceil(inner_circle[2]*expansion_factor))
        outer_circle = detect_outer_circle(image, inner_circle,
                                        range_center, radius_expansion)
    if not outer_circle:
        print('ERROR: Outer iris circle not found!')
        return None, None
    
    if display:
        color_image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
        draw_detected_circles(color_image, inner_circle, outer_circle,
                     range_center, radius_expansion)
        cv2.imshow('iris boundaries', color_image)
        char = cv2.waitKey(0)
        cv2.destroyAllWindows()

    return inner_circle, outer_circle

### Fonction : detect_inner_circle
Détecte le cercle intérieur de l'iris en appliquant des filtres et en ajustant les seuils de manière dynamique pour optimiser la détection des cercles via la transformation de Hough.


In [6]:
def detect_inner_circle(image):
    def extract_edges(processed_image):
        edges = cv2.Canny(processed_image, 20, 100)
        kernel = np.ones((3,3), np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=2)
        blur_size = 2 * random.randrange(5, 11) + 1
        edges = cv2.GaussianBlur(edges, (blur_size, blur_size), 0)
        return edges

    circle_param1 = 200  # High threshold for Canny
    circle_param2 = 120  # Accumulator threshold for circle detection
    candidate_circles = []
    while(circle_param2 > 35 and len(candidate_circles) < 100):
        for median_filter_size, threshold in [(m, t) for m in [3, 5, 7] for t in [20, 25, 30, 35, 40, 45, 50, 55, 60]]:
            # Apply median blur
            blurred = cv2.medianBlur(image, 2 * median_filter_size + 1)

            # Apply threshold
            _, binary_image = cv2.threshold(blurred, threshold, 255, cv2.THRESH_BINARY_INV)

            # Fill contours
            contours, _ = cv2.findContours(binary_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            filled_contours = cv2.drawContours(binary_image, contours, -1, (255), -1)

            # Extract edges
            edges = extract_edges(binary_image)

            # Detect circles using Hough transform
            circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 1, np.array([]), circle_param1, circle_param2)
            if circles is not None and circles.size > 0:
                # Convert the circle parameters to integers
                circles = np.round(circles[0, :]).astype("int")
                for circle in circles:
                    candidate_circles.append(circle)

        circle_param2 = circle_param2 - 1

    if len(candidate_circles) == 0:
        print("No inner circles detected.")
        return None

    return calculate_average_circle(candidate_circles)

### Fonction : calculate_average_circle
Calcule et retourne les coordonnées moyennes et le rayon moyen à partir d'une liste de cercles détectés.


In [ ]:

def calculate_average_circle(circles):
    if not circles:
        return
    average_x = int(np.mean([c[0] for c in circles]))
    average_y = int(np.mean([c[1] for c in circles]))
    average_radius = int(np.mean([c[2] for c in circles]))

    return average_x, average_y, average_radius

### Fonction : identify_optimal_radius
Identifie le rayon optimal en minimisant la distance totale par rapport aux autres rayons dans la liste, permettant une sélection plus précise du cercle.


In [ ]:
def identify_optimal_radius(circle_list):
    optimal_circle = None
    minimum_distance = None
    reference_circles = circle_list[:]
    comparison_circles = circle_list[:]
    for current_circle in reference_circles:
        total_distance = 0
        for compared_circle in comparison_circles:
            total_distance += math.fabs(float(current_circle[2]) - float(compared_circle[2]))
        if not minimum_distance or total_distance < minimum_distance:
            minimum_distance = total_distance
            optimal_circle = current_circle
    return optimal_circle[2]


### Fonction : detect_outer_circle
Détecte dynamiquement le cercle extérieur de l'iris en analysant les bords de l'image traitée. La fonction utilise plusieurs tailles de flou médian et des seuils de détection ajustés itérativement pour identifier les cercles par transformation de Hough. La détection continue jusqu'à ce que le nombre cible de cercles soit atteint ou que les seuils de détection soient épuisés, tout en vérifiant que les cercles détectés se trouvent bien à l'extérieur du cercle intérieur mais dans les limites spécifiées.

In [ ]:

def detect_outer_circle(image, inner_circle, center_range, radius_range):
    def extract_edges(processed_image, upper_threshold):
        lower_threshold = 0  # Fixed low threshold for Canny
        edges = cv2.Canny(processed_image, lower_threshold, upper_threshold, apertureSize=5)
        kernel = np.ones((3,3),np.uint8)
        edges = cv2.dilate(edges, kernel, iterations=1)
        blur_size = 2 * random.randrange(5,11) + 1
        edges = cv2.GaussianBlur(edges,(blur_size,blur_size),0)
        return edges

    def find_circles(hough_threshold, median_sizes, edge_thresholds):
        detected_circles = []
        for median_size, upper_threshold in [(m, t) for m in median_sizes for t in edge_thresholds]:
            # Apply median blur
            median_blurred = cv2.medianBlur(image, 2 * median_size + 1)

            # Extract edges
            edges = extract_edges(median_blurred, upper_threshold)

            # Detect circles using Hough transform
            circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20,
                                       param1=200, param2=hough_threshold, minRadius=0, maxRadius=0)
            if circles is not None and circles.size > 0:
                # Convert the circle parameters to integers
                circles = np.round(circles[0, :]).astype("int")
                for (col, row, radius) in circles:
                    if within_circle(inner_circle[0], inner_circle[1], center_range, col, row) and radius > radius_range:
                        detected_circles.append((col, row, radius))
        return detected_circles

    circle_param2 = 120  # Starting threshold for HoughCircles
    all_circles = []
    while(circle_param2 > 40 and len(all_circles) < 50):
        circles = find_circles(
                        circle_param2, [8,10,12,14,16,18,20], [430,480,530])
        if circles:
            all_circles += circles
        circle_param2 = circle_param2 -1

    if not all_circles:
        print("Alternative strategy for detecting outer iris circle")
        circle_param2 = 120
        while(circle_param2 > 40 and len(all_circles) < 50):
            circles = find_circles(
                            circle_param2, [3,5,7,21,23,25], [430,480,530])
            if circles:
                all_circles += circles
            circle_param2 = circle_param2 -1

    if not all_circles:
        return

    color_image = cv2.cvtColor(image,cv2.COLOR_GRAY2BGR)
    refined_circles = refine_circle_selection(all_circles)

    return calculate_average_circle(refined_circles)

### Fonction : within_circle
Vérifie si un point donné se trouve à l'intérieur du rayon d'un cercle spécifié, en utilisant la distance entre le centre du cercle et le point.


In [ ]:

def within_circle(circle_col, circle_row, circle_radius, point_col, point_row):
    return compute_distance(circle_col, circle_row, point_col, point_row) <= circle_radius

### Fonction : refine_circle_selection
Affine la sélection de cercles détectés en filtrant ceux qui s'écartent significativement des moyennes calculées pour les positions et rayons. cet fonction Utilise des écarts types pour définir les seuils d'acceptation, en éliminant les cercles qui ne correspondent pas aux critères statistiques de cohérence des données.


In [ ]:
def refine_circle_selection(circles):
    if not circles:
        print('Error: No circles found in refine_circle_selection() !')
        return []
    center_x_mean, center_x_dev = compute_standard_deviation([int(i[0]) for i in circles])
    center_y_mean, center_y_dev = compute_standard_deviation([int(i[1]) for i in circles])
    refined = []
    positions = []
    unrefined = []
    deviation_multiplier = 1.5 
    for circle in circles[:]:
        if circle[0] < center_x_mean - deviation_multiplier*center_x_dev or \
           circle[0] > center_x_mean + deviation_multiplier*center_x_dev or \
           circle[1] < center_y_mean - deviation_multiplier*center_y_dev or \
           circle[1] > center_y_mean + deviation_multiplier*center_y_dev:
            unrefined.append(circle)
        else:
            positions.append(circle)
    if len([float(c[2]) for c in positions]) < 3:
        refined = positions
    else:
        optimal_radius = identify_optimal_radius(positions)
        mean_radius, radius_deviation = compute_standard_deviation(
                                    [float(c[2]) for c in positions])
        max_radius = optimal_radius + radius_deviation
        min_radius = optimal_radius - radius_deviation
        for circle in positions:
            if circle[2] < min_radius or \
               circle[2] > max_radius:
                unrefined.append(circle)
            else:
                refined.append(circle)

    return refined

### Fonction : draw_detected_circles
Dessine les cercles détectés sur une image colorée, y compris les cercles internes et externes de l'iris, ainsi que les limites spécifiques de portée et de rayon si elles sont fournies.


In [ ]:

def draw_detected_circles(colored_image, inner_circle, outer_circle,
                 center_range=None, radius_range=None):
    # draw the inner circle
    cv2.circle(colored_image,(inner_circle[0], inner_circle[1]), inner_circle[2],
                     (0,0,255),1)
    # draw the center of the inner circle
    cv2.circle(colored_image,(inner_circle[0],inner_circle[1]),1,(0,0,255),1)
    if center_range:
        # draw outer circle center range limit
        cv2.circle(colored_image,(inner_circle[0], inner_circle[1]), center_range,
                         (0,255,255),1)
    if radius_range:
        # draw outer circle radius range limit
        cv2.circle(colored_image,(inner_circle[0], inner_circle[1]), radius_range,
                         (0,255,255),1)
    # draw the outer circle
    cv2.circle(colored_image, (outer_circle[0], outer_circle[1]), 
               outer_circle[2],(0,255,0),1)
    # draw the center of the outer circle
    cv2.circle(colored_image, (outer_circle[0], outer_circle[1]), 
               1,(0,255,0),1)
    

###  Fonctions Mathématiques Utiles
Ces fonctions fournissent des calculs essentiels pour l'analyse et le traitement d'images:

- **calculate_angle(x1, y1, x2, y2):** Calcule l'angle en degrés entre deux points.
- **compute_distance(x1, y1, x2, y2):** Détermine la distance euclidienne entre deux points.
- **compute_mean(values):** Calcule la moyenne d'une liste de valeurs numériques.
- **compute_median(values):** Trouve la valeur médiane d'une liste, offrant une mesure centrale robuste.
- **compute_standard_deviation(values):** Calcule l'écart type pour mesurer la variation ou la dispersion des valeurs autour de la moyenne.

Ces outils sont cruciaux pour les opérations géométriques et l'analyse statistique dans le traitement des données d'image.


In [ ]:

def calculate_angle(x1, y1, x2, y2):
    return math.degrees(math.atan2(-(y2-y1),(x2-x1)))

def compute_distance(x1, y1, x2, y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def compute_mean(values):
    total = 0.0
    for value in values:
        total += value
    return total/len(values)

def compute_median(values):
    return np.median(np.array(values))

def compute_standard_deviation(values):
    if not values:
        print('Error: empty list parameter in compute_standard_deviation() !')
        return None, None
    mean_value = compute_mean(values)
    sum_of_squares = 0.0
    for value in values:
        sum_of_squares += (value - mean_value) ** 2
    return mean_value, math.sqrt(sum_of_squares/len(values))


### Fonction : IrisNormalization
Normalise la région de l'iris en convertissant la région annulaire entre les cercles interne et externe de l'iris en une image rectangulaire de dimensions fixes. Cette fonction crée une cartographie du contour de l'iris à une image de 64x512 pixels, utilisant l'interpolation pour ajuster les données des pixels entre les frontières interne et externe. Cela facilite une comparaison plus uniforme et précise des caractéristiques de l'iris dans des images de reconnaissance.

**Paramètres:**
- `image`: Image d'entrée contenant l'iris.
- `inner_circle`: Coordonnées et rayon du cercle interne de l'iris.
- `outer_circle`: Coordonnées et rayon du cercle externe de l'iris.

**Retourne:**
- `res_image`: Image de l'iris normalisée avec des dimensions de 64x512 pixels, prête pour l'analyse et la reconnaissance.


In [ ]:
def IrisNormalization(image,inner_circle,outer_circle ):
    localized_img=image
    row=64
    col=512
    normalized_iris=np.zeros(shape=(64,512))
    inner_y=inner_circle[0]  #height
    inner_x=inner_circle[1]  #width
    outer_y=outer_circle[0]
    outer_x=outer_circle[1]
    angle=2.0*math.pi/col
    inner_boundary_x = np.zeros(shape=(1,col))
    inner_boundary_y = np.zeros(shape=(1,col))
    outer_boundary_x = np.zeros(shape=(1,col))
    outer_boundary_y = np.zeros(shape=(1,col))
    for j in range(col):


        inner_boundary_x[0][j]=inner_circle[0]+inner_circle[2]*math.cos(angle*(j))
        inner_boundary_y[0][j]=inner_circle[1]+inner_circle[2]*math.sin(angle*(j))
        
        outer_boundary_x[0][j]=outer_circle[0]+outer_circle[2]*math.cos(angle*(j))
        outer_boundary_y[0][j]=outer_circle[1]+outer_circle[2]*math.sin(angle*(j))
        
    for j in range (512):
        for i in range (64):
             normalized_iris[i][j]=localized_img[min(int(int(inner_boundary_y[0][j])
                                   +(int(outer_boundary_y[0][j])-int(inner_boundary_y[0][j]))*(i/64.0)),localized_img.shape[0]-1)][min(int(int(inner_boundary_x[0][j])
                                   +(int(outer_boundary_x[0][j])-int(inner_boundary_x[0][j]))
                                   *(i/64.0)),localized_img.shape[1]-1)]

    res_image=255-normalized_iris
    return res_image

### Fonction : ImageEnhancement
Améliore l'image normalisée de l'iris pour augmenter la clarté et le contraste des caractéristiques importantes. La fonction convertit l'image en entier 8 bits, applique une égalisation d'histogramme locale utilisant un élément structurant en forme de disque pour mieux répartir l'intensité lumineuse, et extrait ensuite une région d'intérêt (ROI) pour réduire les distractions et se concentrer sur les parties les plus pertinentes de l'iris.

**Paramètres:**
- `normalized_iris`: Image de l'iris normalisée préparée pour l'amélioration.

**Retourne:**
- `roi`: Région d'intérêt de l'image améliorée, plus adaptée à l'analyse ultérieure.


In [ ]:
def ImageEnhancement(normalized_iris):
    row=64
    col=512
    normalized_iris = normalized_iris.astype(np.uint8)
    
    
    enhanced_image=normalized_iris
     
    enhanced_image = equalize(enhanced_image, disk(32))
    
    roi = enhanced_image[0:48,:]
    return roi

### Fonction : preprocess
Traite une image pour l'analyse de l'iris, en commençant par la lecture de l'image, la conversion en niveaux de gris, la détection des contours de l'iris et de la pupille, la normalisation de la région de l'iris, et enfin l'amélioration de cette région pour une analyse détaillée. La fonction visualise également les contours détectés sur l'image initiale pour vérification, puis extrait et améliore la région normalisée pour obtenir une région d'intérêt (ROI) optimale pour l'extraction de caractéristiques.

**Processus:**
1. **Lecture et conversion:** L'image est lue et convertie en niveaux de gris.
2. **Détection des contours:** Utilise `locate_iris_boundaries` pour identifier les cercles de l'iris et de la pupille.
3. **Visualisation des contours:** Dessine les contours de l'iris et de la pupille sur l'image pour vérification.
4. **Normalisation de l'iris:** Applique `IrisNormalization` pour transformer la région annulaire de l'iris en une forme rectangulaire standard.
5. **Amélioration de l'image:** Utilise `ImageEnhancement` pour améliorer la clarté et le contraste de l'image normalisée.

**Paramètres:**
- `image`: Chemin de l'image à traiter.

**Retourne:**
- `ROI`: Région d'intérêt de l'image améliorée, prête pour l'extraction de caractéristiques et l'analyse.


In [ ]:
def preprocess(image):
    imagep = cv2.imread(image)                
    eye = cv2.cvtColor(imagep, cv2.COLOR_BGR2GRAY)
    outer , inner = locate_iris_boundaries(eye)
    x_iris,y_iris,r_iris = outer
    x_pupil,y1_pupil,r1_pupil = inner
    dra1 = eye.copy()
    cv2.circle(dra1, (x_iris, y_iris), r_iris, color=(255, 0, 0), thickness=3)
    cv2.circle(dra1, (x_pupil, y1_pupil), r1_pupil, color=(255, 0, 0), thickness=3)
    iris, pupil = outer,inner
    normalized = IrisNormalization(eye, pupil, iris)
    ROI = ImageEnhancement(normalized)
    return ROI
    
    


 

## exemple de l'utilisation de preprocess

In [ ]:
roi = preprocess("Temp\S6030S07.jpg")

## affichage de l'exemple

In [ ]:
cv2.imwrite('image.png', roi)

### Fonction : savepreprocessed
Enregistre une image prétraitée dans un répertoire spécifié, créant le chemin de dossier si nécessaire. Cette fonction est utile pour organiser et stocker des images traitées dans une structure de répertoire ordonnée, facilitant l'accès et la gestion des données d'image dans des projets plus vastes.


In [ ]:
def savepreprocessed(image, folder, image_name):
    path_preprocessed = 'Backend/0_Data/Preprocessed_Data'
    if not os.path.exists(os.path.join(path_preprocessed, folder)):
        os.makedirs(os.path.join(path_preprocessed, folder))
    cv2.imwrite(os.path.join(path_preprocessed, folder, image_name + '.jpg'), image)

### Traitement en lot et sauvegarde des images prétraitées

Ce bloc de code parcourt un répertoire contenant des images brutes, les traite une par une, puis sauvegarde les résultats dans un répertoire approprié. Chaque image est lue, traitée via la fonction `preprocess`, et ensuite sauvegardée avec son nom d'origine (sans l'extension) dans un sous-dossier spécifique qui correspond à son dossier d'origine.

**Détails du processus:**
1. **Parcours des dossiers:** Le code parcourt chaque dossier dans le répertoire spécifié 'Backend/0_Data/Raw_Data/'.
2. **Traitement des images:** Chaque image dans ces dossiers est traitée pour normaliser et améliorer les régions de l'iris.
3. **Sauvegarde des images traitées:** Les images traitées sont sauvegardées dans une structure de dossier qui reflète leur organisation d'origine, aidant à maintenir une bonne gestion des données pour des analyses ultérieures.

Ce processus automatise la préparation des images pour des systèmes de reconnaissance d'iris, assurant que toutes les images sont formatées de manière uniforme et stockées de manière organisée.


In [ ]:
row_data = 'Backend/0_Data/Raw_Data/'

for folder in os.listdir(row_data):
    folder_path = os.path.join(row_data, folder)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        preprocessed_image = preprocess(image_path)
        savepreprocessed(preprocessed_image, folder, os.path.splitext(image_name)[0])


### Description des fonctions de traitement d'image pour l'extraction de caractéristiques

Ce code implémente plusieurs fonctions pour le traitement d'image, la modulation, et l'extraction de caractéristiques basées sur des filtres spatiaux, notamment des filtres de Gabor.

1. **Fonctions de modulation et de filtrage Gabor :**
   - `m(x, y, f)`: Module une valeur basée sur la fréquence et la position, utilisée pour créer des filtres de Gabor.
   - `gabor(x, y, dx, dy, f)`: Calcule un filtre de Gabor en utilisant la modulation précédemment définie, caractérisé par des paramètres de dispersion `dx` et `dy` et une fréquence `f`.

2. **Fonction pour calculer les filtres spatiaux :**
   - `spatial(f, dx, dy)`: Génère un filtre spatial de 4x4 en utilisant la fonction `gabor`, adapté pour un traitement par blocs sur des images.

3. **Extraction de vecteur de caractéristiques :**
   - `get_vec(convolvedtrain1, convolvedtrain2)`: Extrait un vecteur de caractéristiques de deux images filtrées, en calculant la moyenne et l'écart-type des valeurs absolues sur des blocs de 4x4 pixels.

4. **Extraction de caractéristiques globale :**
   - `FeatureExtraction(enhanced)`: Applique les filtres spatiaux sur une région définie de l'image améliorée, extrait les vecteurs de caractéristiques pour chaque image dans un ensemble, et retourne un ensemble de vecteurs pour analyse ou apprentissage ultérieur.

Chaque image est d'abord traitée pour extraire une région d'intérêt, puis convoluée avec deux filtres de Gabor différents. Les résultats sont utilisés pour construire des vecteurs de caractéristiques qui sont ensuite agrégés pour former un ensemble de données pour l'entraînement et le test dans des le modèles de reconnaissance.


In [ ]:
import cv2
import numpy as np
import glob
import math
import scipy
from scipy.spatial import distance
from scipy import signal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics


def m(x ,y, f):
    val = np.cos(2*np.pi*f*math.sqrt(x **2 + y**2))
    return val
#spatial filter as defined in paper
def gabor(x, y, dx, dy, f):
    gb = (1/(2*math.pi*dx*dy))*np.exp(-0.5*(x**2 / dx**2 + y**2 / dy**2)) * m(x, y, f)
    return gb

#function to calculate spatial filter over 4x4 blocks
def spatial(f,dx,dy):
    sfilter=np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            sfilter[i,j]=gabor((-2+j),(-2+i),dx,dy,f)
    return sfilter

def get_vec(convolvedtrain1,convolvedtrain2):
    feature_vec=[]
    for i in range(12):
            for j in range(128):
                #Run 4 by 4 filtered block iteratively over the entire image
                start_height = i*4
                end_height = start_height+4
                start_wid = j*4
                end_wid = start_wid+4
                grid1 = convolvedtrain1[start_height:end_height, start_wid:end_wid]
                grid2 = convolvedtrain2[start_height:end_height, start_wid:end_wid]

                # Channel 1
                absolute = np.absolute(grid1)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

                # Channel 2
                absolute = np.absolute(grid2)
                # mean
                mean = np.mean(absolute)
                feature_vec.append(mean)
                #deviation
                std = np.mean(np.absolute(absolute-mean))
                feature_vec.append(std)

    return feature_vec

def FeatureExtraction(enhanced):
    con1=[]
    con2=[]
    #get spatial filters
    filter1=spatial(0.67,3,1.5)
    filter2=spatial(0.67,4,1.5) 
    
    feature_vector=[] #
    
    for i in range(len(enhanced)):
        img=enhanced[i]
        #define a 48x512 region over which the filters are applied
        img_roi=img[0:48,:]
        
        filtered1=scipy.signal.convolve2d(img_roi,filter1,mode='same')
        filtered2=scipy.signal.convolve2d(img_roi,filter2,mode='same')
        
        con1.append(filtered1)
        con2.append(filtered2)
        fv=get_vec(filtered1,filtered2)
        feature_vector.append(fv)
    return feature_vector

### Description du processus de chargement des données prétraitées

Ce script parcourt un répertoire contenant des images prétraitées, les charge et les convertit en niveaux de gris pour préparer des données d'entraînement et de test pour le réseau de neurones . Il stocke également les étiquettes associées à chaque image, basées sur le nom du dossier, ce qui est utile pour des tâches de classification.


In [ ]:
preprocessed_data = 'Backend/A_Data/Preprocessed_Data'
label = []
data = []
for folder in os.listdir(preprocessed_data):  
      folder_path = os.path.join(preprocessed_data, folder)
      for image_preprocessed in os.listdir(folder_path):
        input_preprocessedimg = cv2.imread(os.path.join(folder_path,image_preprocessed))
        img= cv2.cvtColor(input_preprocessedimg, cv2.COLOR_BGR2GRAY)
        label.append(folder)
        data.append(img)
        

### Description de l'application de l'extraction de caractéristiques

Ce fragment de code utilise la fonction `FeatureExtraction` pour traiter une liste d'images et en extraire des caractéristiques. Cette étape est cruciale pour transformer les données visuelles brutes en un format plus adapté à l'apprentissage automatique.


In [ ]:
new_data = []
new_data = FeatureExtraction(data)


### Conversion des données et étiquettes en tableaux NumPy

Ce fragment de code convertit les listes `label` et `new_data` en tableaux NumPy, facilitant ainsi leur manipulation et utilisation dans le réseau de neurones.


In [ ]:
f1 = np.array(label)
f2 = np.array(new_data)


### Configuration d'un réseau de neurones pour la classification avec Keras

Ce script configure et compile un modèle de réseau de neurones profond pour classer des images basées sur leurs vecteurs de caractéristiques. Il utilise Keras, une bibliothèque de deep learning populaire, pour construire et entraîner le modèle.

**Processus de configuration :**
1. **Préparation des données :**
   - `x` et `y_` sont initialisés avec les tableaux `f2` et `f1` respectivement.
   - Les étiquettes `y_` sont converties en format catégorique (`y`), utilisant `to_categorical` pour l'apprentissage supervisé.
   - Les données sont divisées en ensembles d'entraînement et de test avec `train_test_split`, répartissant 20% des données pour le test.

2. **Construction du modèle :**
   - Le modèle utilise une architecture `Sequential` avec trois couches `Dense`. La première couche a 1024 unités, la seconde 2048, et la troisième (couche de sortie) 65 unités correspondant aux classes potentielles, toutes avec l'activation `relu` sauf la dernière qui utilise `softmax` pour la classification.
   - Le modèle est compilé avec l'optimiseur `SGD` avec un momentum de 0.9, utilisant la `categorical_crossentropy` comme fonction de perte et mesurant `accuracy`.

**Visualisation de la configuration du modèle :**
- Un résumé du modèle est affiché pour vérifier sa structure et ses paramètres.

In [ ]:
import numpy as np
from keras.utils import to_categorical
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.constraints import max_norm
from keras.layers import Dropout

x = f2
y_ = f1
y = to_categorical(y_)  

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)


model = Sequential()
model.add(Dense(1024, input_shape=(6144,), activation='relu', name='fc1'))
model.add(Dense(2048, activation='relu', name='fc2'))
model.add(Dense(65, activation='softmax', name='output'))


optimizer = SGD(momentum=0.9)

# Compile the model
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('Neural Network Model Summary: ')
print(model.summary())


### Entraînement du modèle de réseau de neurones

Ce script lance l'entraînement du modèle de réseau de neurones configuré précédemment. Il utilise les données divisées en ensembles d'entraînement et de test pour optimiser et évaluer le modèle au cours de plusieurs itérations.


In [ ]:
model.fit(train_x, train_y,validation_data=(test_x,test_y), verbose=1, epochs=100)


### Visualisation de la précision d'entraînement et de test

Ce script génère un graphique pour visualiser la progression de la précision du modèle au cours de l'entraînement et de la validation. Ce graphique aide à identifier visuellement comment le modèle s'améliore avec chaque époque et à détecter des signes de sur-entraînement ou de sous-entraînement.


In [ ]:
train_loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
train_acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']
xc = range(100)

plt.figure(figsize=(20,10))
plt.plot(xc,train_acc,label = 'Train accuracy')
plt.plot(xc,val_acc,label='Test accuracy')
plt.legend(loc='upper left',prop={'size':20})
plt.title('Train and Test Accuracy plot',size=20)
plt.xlabel('Epochs',size=20)
plt.ylabel('Accuracy',size=20)

## enregistrer le modèle

In [ ]:
# Sauvegarder le modèle
model.save('Gabor+NN.h5')

print("Le modèle a été sauvegardé avec succès !")

### Utilisation d'un modèle entraîné pour prédire la classe d'une image

Ce script illustre comment charger et utiliser un modèle de réseau de neurones convolutif préalablement entraîné pour classer une image spécifique. Il démontre le processus complet de la charge du modèle, de la préparation de l'image, de l'extraction des caractéristiques, et de la prédiction de la classe.


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Charger le modèle entraîné
model = load_model('3_Model/Gabor_NN.h5')

# Charger l'image à partir du dossier de normalisation
image_path = 'Backend/0_Data\Preprocessed_Data/009\S6009S01.jpg'
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
print(img.shape)
# Extraire les caractéristiques de l'image
features = FeatureExtraction([img])

prediction = model.predict(np.array(features))

# Trouver l'index de la classe avec la probabilité maximale
predicted_class_index = np.argmax(prediction)

# Afficher la classe prédite
print("Classe prédite :", predicted_class_index)
